<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/html_to_csv_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
import openai
import re
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_html = '/content/drive/MyDrive/Dissertation/Study 1/googleData/html/Chrome/takeout1_chrome_MyActivity.html'

In [ ]:
import re

def extract_and_remove_datetime(text):
    # Define a regex pattern for the specified date and time format
    datetime_pattern = r"(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))\s+(?P<day>\d{1,2}),\s+(?P<year>\d{4}),\s+(?P<hour>\d{1,2}):(?P<minute>\d{2}):(?P<second>\d{2})\s+(?P<meridian>[APap][Mm])\s+(?P<timezone>[A-Z]+)"

    # Check if the pattern is found in the text
    match = re.search(datetime_pattern, text)

    if match:
        # Extract date and time components using named groups
        date = match.group('month') + " " + match.group('day') + ", " + match.group('year')
        time = match.group('hour') + ":" + match.group('minute') + ":" + match.group('second') + " " + match.group('meridian') + " " + match.group('timezone')

        # Remove the extracted date and time from the original text
        # cleaned_text = re.sub(datetime_pattern, "", text)

        return date, time
    else:
        return None, None, text  # Return original text if not found

In [ ]:
def read_html_file_fast(input_html):
    """Efficiently reads an HTML file and returns a parsed BeautifulSoup object."""
    with open(input_html, 'r', encoding='utf-8') as f:
        return BeautifulSoup(f.read(), 'lxml')  # Use 'lxml' for speed

soup = read_html_file_fast(input_html)

In [ ]:
output_filename = os.path.splitext(os.path.basename(input_html))[0]

In [ ]:
output_filepath = os.path.join(os.path.dirname('/content/drive/MyDrive/Dissertation/Study 1/googleData/CSV_data'), f"{output_filename}.csv")

In [ ]:
# Extract all activity entries
entries = soup.find_all('div', class_='outer-cell')

data = []
for entry in entries:
    # Extract source (Column 1)
    source_tag = entry.find('p', class_='mdl-typography--title')
    source = source_tag.get_text(strip=True) if source_tag else None

    # Extract link (Column 2)
    link_tag = entry.find('a')
    link = None
    if link_tag:
        match = re.search(r'q=(https://[^\&]+)', link_tag['href'])
        link = match.group(1) if match else link_tag['href']

    # Extract title (Column 3)
    title = None if "https://" in link_tag.get_text(strip=True) else link_tag.get_text(strip=True)

    # Extract date and time (Column 4 & 5)
    text_content = entry.find('div', class_='content-cell').get_text(strip=True)
    # date_time_match = re.search(r'(\w+ \d{1,2}, \d{4}), (\d{1,2}:\d{2}:\d{2} [APM]+ [A-Z]+)', text_content)
    date_time_match = extract_and_remove_datetime(text_content)

    # Access the elements of the tuple returned by extract_and_remove_datetime
    date = date_time_match[0] if date_time_match[0] is not None else None # Access the first element for date
    time = date_time_match[1] if date_time_match[1] is not None else None # Access the second element for time

    # Append data to list
    data.append([source, link, title, date, time])

# Create DataFrame
df = pd.DataFrame(data, columns=['Search Site', 'Search Link', 'Search Title', 'Search Date', 'Search Time'])

# Save to CSV
# df.to_csv(output_filepath, index=False)